In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'numpy'

In [2]:
training = pd.read_csv("titanic/train.csv")
test = pd.read_csv("titanic/test.csv")

training["train_test"] = 1
test["train_test"] = 0
test["Survived"] = np.NaN
all_data = pd.concat([training,test])

%matplotlib inline
all_data.columns

NameError: name 'pd' is not defined

In [3]:
#quick look at our data types & null counts
training.info()

NameError: name 'training' is not defined

In [4]:
# understading the central tendencies of the data
training.describe()

NameError: name 'training' is not defined

In [5]:
# seperate numeric columns
training.describe().columns

NameError: name 'training' is not defined

In [6]:
# looking at numeric and categorical values seperately
df_num = training[["Age", "SibSp", "Parch", "Fare"]]
df_cat = training[["Survived", "Pclass", "Sex", "Ticket", "Cabin", "Embarked"]]

NameError: name 'training' is not defined

In [7]:
# distribution of all numerical variables
for i in df_num.columns:
    plt.hist(df_num[i])
    plt.title(i)
    plt.show()

NameError: name 'df_num' is not defined

In [8]:
print(df_num.corr())
sns.heatmap(df_num.corr())

NameError: name 'df_num' is not defined

In [9]:
# compare survival rate across Age, SibSp, Parch and Fare
pd.pivot_table(training, index="Survived", values=["Age", "SibSp", "Parch", "Fare"])

NameError: name 'pd' is not defined

In [10]:
for i in df_cat.columns:
    sns.barplot(x=df_cat[i].value_counts().index, y=df_cat[i].value_counts()).set_title(i)
    plt.show()

NameError: name 'df_cat' is not defined

In [11]:
# comparing survival and each of these categorical variables
print(pd.pivot_table(training, index="Survived", columns="Pclass", values="Ticket", aggfunc="count"))
print()
print(pd.pivot_table(training, index="Survived", columns="Sex", values="Ticket", aggfunc="count"))
print()
print(pd.pivot_table(training, index="Survived", columns="Embarked", values="Ticket", aggfunc="count"))
print()


NameError: name 'pd' is not defined

## Feature Engineering 
1. Cabin
2. Tickets
3. Does a person's title relater to survival rate?



In [12]:
df_cat.Cabin
training["cabin_multiple"] = training.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(" ")))

training["cabin_multiple"].value_counts()

NameError: name 'df_cat' is not defined

In [13]:
pd.pivot_table(training, index="Survived", columns="cabin_multiple", values="Ticket", aggfunc="count")

NameError: name 'pd' is not defined

In [14]:
training["cabin_adv"] = training.Cabin.apply(lambda x: str(x)[0])

NameError: name 'training' is not defined

In [15]:
print(training.cabin_adv.value_counts())
pd.pivot_table(training, index="Survived", columns="cabin_adv", values="Name", aggfunc="count")

NameError: name 'training' is not defined

In [16]:
# understand ticket values better
# numeric vs non numeric
training["numeric_ticket"] = training.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
training["ticket_letters"] = training.Ticket.apply(lambda x: "".join(x.split(" ")[:-1]).replace(".", "").replace("/", "").lower() if len(x.split(" ")[:-1]) > 0 else 0)

NameError: name 'training' is not defined

In [17]:
training["numeric_ticket"].value_counts()

NameError: name 'training' is not defined

In [18]:
training["ticket_letters"].value_counts()

NameError: name 'training' is not defined

In [19]:
# difference in numeric vs non-numeric tickets in survival rate
pd.pivot_table(training, index="Survived", columns="numeric_ticket", values="Ticket", aggfunc="count")

NameError: name 'pd' is not defined

In [20]:
# survival rate across different ticket type
pd.pivot_table(training, index="Survived", columns="ticket_letters", values="Ticket", aggfunc="count")

NameError: name 'pd' is not defined

In [21]:
# feature engineering on person's title 
training.Name.head(50)
training["name_title"] = training.Name.apply(lambda x: x.split(",")[1].split(".")[0].strip())

NameError: name 'training' is not defined

In [22]:
training["name_title"].value_counts()

NameError: name 'training' is not defined

## Data Preprocessing for Model

In [23]:
# create all categorical variable
all_data["cabin_multiple"] = all_data.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(" ")))
all_data["cabin_adv"] = all_data.Cabin.apply(lambda x: str(x)[0])
all_data["numeric_ticket"] = all_data.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
all_data["ticket_letter"] = all_data.Ticket.apply(lambda x: "".join(x.split(" ")[:-1]).replace(".", "").replace("/", "").lower() if len(x.split(" ")[:-1]) > 0 else 0)
all_data["name_title"] = all_data.Name.apply(lambda x: x.split(",")[1].split(".")[0].strip())

# impute null for continous data
all_data.Age = all_data.Age.fillna(training.Age.mean())
all_data.Fare = all_data.Fare.fillna(training.Fare.mean())

# drop null "embarked" rows. Only 2 instances of this in training and 0 in test
all_data.dropna(subset=["Embarked"], inplace=True)

# tried log norm of sibsp (not used)
all_data["norm_sibsp"] = np.log(all_data.SibSp+1)
all_data["norm_sibsp"].hist()

# log norm of fare(used)
all_data["norm_fare"] = np.log(all_data.Fare+1)
all_data["norm_fare"].hist()

# converted fare to category for pd.get_dummies()
all_data.Pclass = all_data.Pclass.astype(str)

# created dummy variables from categories (also can use OneHotEncoder)
all_dummies = pd.get_dummies(all_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "norm_fare", "Embarked", "cabin_adv", "cabin_multiple", "numeric_ticket", "name_title", "train_test"]])

# split to train test again 
X_train = all_dummies[all_dummies.train_test == 1].drop(["train_test"], axis=1)
X_test = all_dummies[all_dummies.train_test == 0].drop(["train_test"], axis=1)

y_train = all_data[all_data.train_test==1].Survived
y_train.shape



NameError: name 'all_data' is not defined

In [24]:
# scale data
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
all_dummies_scaled = all_dummies.copy()
all_dummies_scaled[["Age", "SibSp", "Parch", "norm_fare"]] = scale.fit_transform(all_dummies_scaled[["Age", "SibSp", "Parch", "norm_fare"]])

all_dummies_scaled

X_train_scaled = all_dummies_scaled[all_dummies_scaled.train_test == 1].drop(["train_test"], axis=1)
X_test_scaled = all_dummies_scaled[all_dummies_scaled.train_test == 0].drop(["train_test"], axis=1)

y_train = all_data[all_data.train_test == 1].Survived

ModuleNotFoundError: No module named 'sklearn'

In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

ModuleNotFoundError: No module named 'sklearn'

In [26]:
gnb = GaussianNB()
cv = cross_val_score(gnb, X_train_scaled, y_train, cv=5)
print(cv)
print(cv.mean())

NameError: name 'GaussianNB' is not defined

In [27]:
lr = LogisticRegression(max_iter = 2000)
cv = cross_val_score(lr, X_train_scaled, y_train, cv=5)
print(cv)
print(cv.mean())

NameError: name 'LogisticRegression' is not defined

In [28]:
lr = LogisticRegression(max_iter = 2000)
cv = cross_val_score(lr, X_train, y_train, cv=5)
print(cv)
print(cv.mean())

NameError: name 'LogisticRegression' is not defined

In [29]:
dt = tree.DecisionTreeClassifier(random_state=1)
cv = cross_val_score(dt, X_train, y_train, cv=5)
print(cv)
print(cv.mean())

NameError: name 'tree' is not defined

In [30]:
dt = tree.DecisionTreeClassifier(random_state=1)
cv = cross_val_score(dt, X_train_scaled, y_train, cv=5)
print(cv)
print(cv.mean())

NameError: name 'tree' is not defined

In [31]:
knn = KNeighborsClassifier()
cv = cross_val_score(knn, X_train_scaled, y_train, cv=5)
print(cv)
print(cv.mean())

NameError: name 'KNeighborsClassifier' is not defined

In [32]:
knn = KNeighborsClassifier()
cv = cross_val_score(knn, X_train, y_train, cv=5)
print(cv)
print(cv.mean())

NameError: name 'KNeighborsClassifier' is not defined

In [33]:
rf = RandomForestClassifier(random_state=1)
cv = cross_val_score(rf, X_train, y_train, cv=5)
print(cv)
print(cv.mean())

NameError: name 'RandomForestClassifier' is not defined

In [34]:
rf = RandomForestClassifier(random_state=1)
cv = cross_val_score(rf, X_train_scaled, y_train, cv=5)
print(cv)
print(cv.mean())

NameError: name 'RandomForestClassifier' is not defined

In [35]:
svc = SVC(probability=True)
cv = cross_val_score(svc, X_train_scaled, y_train, cv=5)
print(cv)
print(cv.mean())

NameError: name 'SVC' is not defined

In [36]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state=1)
cv = cross_val_score(xgb, X_train_scaled, y_train, cv=5)
print(cv)
print(cv.mean())

ModuleNotFoundError: No module named 'xgboost'

In [37]:
# voting classifier
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators = [("lr", lr), ("knn", knn), ("rf", rf), ("gnb", gnb), ("svc", svc), ("xgb", xgb)], voting="soft")

ModuleNotFoundError: No module named 'sklearn'

In [38]:
cv = cross_val_score(voting_clf, X_train_scaled, y_train, cv=5)
print(cv)
print(cv.mean())

NameError: name 'cross_val_score' is not defined

In [39]:
voting_clf.fit(X_train_scaled, y_train)
y_hat_base_vc = voting_clf.predict(X_test_scaled)
basic_submission = {"PassengerId": test.PassengerId, "Surrvived": y_hat_base_vc}
base_submission = pd.DataFrame(data=basic_submission)
base_submission.to_csv("base_submission.csv", index=False)


NameError: name 'voting_clf' is not defined

## Model Tuning

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

ModuleNotFoundError: No module named 'sklearn'

In [41]:
def clf_performance(classifier, model_name):
    print(model_name)
    print("Best Score: "+ str(classifier.best_score_))
    print("Best Parameters: "+ str(classifier.best_params_))

In [42]:
lr = LogisticRegression()
param_grid = {"max_iter": [2000],
              "penalty": ["l1", "l2"],
              "C": np.logspace(-4, 4, 20),
              "solver": ["liblinear"]}

clf_lr = GridSearchCV(lr, param_grid=param_grid, cv=5, verbose=True, n_jobs=-1)
best_clf_lr = clf_lr.fit(X_train_scaled, y_train)
clf_performance(best_clf_lr, "Logistic Regression")

NameError: name 'LogisticRegression' is not defined

In [43]:
knn = KNeighborsClassifier()
param_grid = {"n_neighbors": [3, 5, 7, 9],
              "weights": ["uniform", "distance"],
              "algorithm": ["auto", "ball_tree", "kd_tree"],
              "p": [1, 2]}

clf_knn = GridSearchCV(knn, param_grid=param_grid, cv=5, verbose=True, n_jobs=-1)
best_clf_knn = clf_knn.fit(X_train_scaled, y_train)
clf_performance(best_clf_knn, "KNN")

NameError: name 'KNeighborsClassifier' is not defined

In [44]:
svc = SVC(probability=True)
param_grid = tuned_parameters = [{"kernel": ["rbf"], "gamma": [.1, .5, 1, 2, 5, 10],
                                  "C": [.1, 1, 10, 100, 1000]},
                                  {"kernel": ["linear"], "C": [.1, 1, 10, 100, 1000]},
                                  {"kernel": ["poly"], "degree": [2, 3, 4, 5], "C": [.1, 1, 10, 100, 1000]}]

clf_svc = GridSearchCV(svc, param_grid=param_grid, cv=5, verbose=True, n_jobs=-1)
best_clf_svc = clf_svc.fit(X_train_scaled, y_train)
clf_performance(best_clf_svc, "SVC")

NameError: name 'SVC' is not defined

In [45]:
rf = RandomForestClassifier(random_state = 1)
param_grid =  {'n_estimators': [400,450,500,550],
               'criterion':['gini','entropy'],
                                  'bootstrap': [True],
                                  'max_depth': [15, 20, 25],
                                  'max_features': ['auto','sqrt', 10],
                                  'min_samples_leaf': [2,3],
                                  'min_samples_split': [2,3]}
                                  
clf_rf = GridSearchCV(rf, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_rf = clf_rf.fit(X_train_scaled,y_train)
clf_performance(best_clf_rf,'Random Forest')

NameError: name 'RandomForestClassifier' is not defined

In [46]:
best_rf = best_clf_rf.best_estimator_.fit(X_train_scaled,y_train)
feat_importances = pd.Series(best_rf.feature_importances_, index=X_train_scaled.columns)
feat_importances.nlargest(20).plot(kind='barh')

NameError: name 'best_clf_rf' is not defined

In [47]:
xgb = XGBClassifier(random_state = 1)

param_grid = {
    'n_estimators': [450,500,550],
    'colsample_bytree': [0.75,0.8,0.85],
    'max_depth': [None],
    'reg_alpha': [1],
    'reg_lambda': [2, 5, 10],
    'subsample': [0.55, 0.6, .65],
    'learning_rate':[0.5],
    'gamma':[.5,1,2],
    'min_child_weight':[0.01],
    'sampling_method': ['uniform']
}

clf_xgb = GridSearchCV(xgb, param_grid = param_grid, cv = 5, verbose = True, n_jobs = -1)
best_clf_xgb = clf_xgb.fit(X_train_scaled,y_train)
clf_performance(best_clf_xgb,'XGB')

NameError: name 'XGBClassifier' is not defined

In [48]:
y_hat_xgb = best_clf_xgb.best_estimator_.predict(X_test_scaled).astype(int)
xgb_submission = {'PassengerId': test.PassengerId, 'Survived': y_hat_xgb}
submission_xgb = pd.DataFrame(data=xgb_submission)
submission_xgb.to_csv('xgb_submission3.csv', index=False)

NameError: name 'best_clf_xgb' is not defined

## Model ensemble

In [49]:
best_lr = best_clf_lr.best_estimator_
best_knn = best_clf_knn.best_estimator_
best_svc = best_clf_svc.best_estimator_
best_rf = best_clf_rf.best_estimator_
best_xgb = best_clf_xgb.best_estimator_

voting_clf_hard = VotingClassifier(estimators = [('knn',best_knn),('rf',best_rf),('svc',best_svc)], voting = 'hard') 
voting_clf_soft = VotingClassifier(estimators = [('knn',best_knn),('rf',best_rf),('svc',best_svc)], voting = 'soft') 
voting_clf_all = VotingClassifier(estimators = [('knn',best_knn),('rf',best_rf),('svc',best_svc), ('lr', best_lr)], voting = 'soft') 
voting_clf_xgb = VotingClassifier(estimators = [('knn',best_knn),('rf',best_rf),('svc',best_svc), ('xgb', best_xgb),('lr', best_lr)], voting = 'soft')

print('voting_clf_hard :',cross_val_score(voting_clf_hard,X_train,y_train,cv=5))
print('voting_clf_hard mean :',cross_val_score(voting_clf_hard,X_train,y_train,cv=5).mean())

print('voting_clf_soft :',cross_val_score(voting_clf_soft,X_train,y_train,cv=5))
print('voting_clf_soft mean :',cross_val_score(voting_clf_soft,X_train,y_train,cv=5).mean())

print('voting_clf_all :',cross_val_score(voting_clf_all,X_train,y_train,cv=5))
print('voting_clf_all mean :',cross_val_score(voting_clf_all,X_train,y_train,cv=5).mean())

print('voting_clf_xgb :',cross_val_score(voting_clf_xgb,X_train,y_train,cv=5))
print('voting_clf_xgb mean :',cross_val_score(voting_clf_xgb,X_train,y_train,cv=5).mean())

NameError: name 'best_clf_lr' is not defined

In [50]:
#in a soft voting classifier we can weight some models more than others. I used a grid search to explore different weightings
#no new results here
params = {'weights' : [[1,1,1],[1,2,1],[1,1,2],[2,1,1],[2,2,1],[1,2,2],[2,1,2]]}

vote_weight = GridSearchCV(voting_clf_soft, param_grid = params, cv = 5, verbose = True, n_jobs = -1)
best_clf_weight = vote_weight.fit(X_train_scaled,y_train)
clf_performance(best_clf_weight,'VC Weights')
voting_clf_sub = best_clf_weight.best_estimator_.predict(X_test_scaled)

NameError: name 'GridSearchCV' is not defined

In [51]:
#Make Predictions 
voting_clf_hard.fit(X_train_scaled, y_train)
voting_clf_soft.fit(X_train_scaled, y_train)
voting_clf_all.fit(X_train_scaled, y_train)
voting_clf_xgb.fit(X_train_scaled, y_train)

best_rf.fit(X_train_scaled, y_train)
y_hat_vc_hard = voting_clf_hard.predict(X_test_scaled).astype(int)
y_hat_rf = best_rf.predict(X_test_scaled).astype(int)
y_hat_vc_soft =  voting_clf_soft.predict(X_test_scaled).astype(int)
y_hat_vc_all = voting_clf_all.predict(X_test_scaled).astype(int)
y_hat_vc_xgb = voting_clf_xgb.predict(X_test_scaled).astype(int)

NameError: name 'voting_clf_hard' is not defined

In [52]:
#convert output to dataframe 
final_data = {'PassengerId': test.PassengerId, 'Survived': y_hat_rf}
submission = pd.DataFrame(data=final_data)

final_data_2 = {'PassengerId': test.PassengerId, 'Survived': y_hat_vc_hard}
submission_2 = pd.DataFrame(data=final_data_2)

final_data_3 = {'PassengerId': test.PassengerId, 'Survived': y_hat_vc_soft}
submission_3 = pd.DataFrame(data=final_data_3)

final_data_4 = {'PassengerId': test.PassengerId, 'Survived': y_hat_vc_all}
submission_4 = pd.DataFrame(data=final_data_4)

final_data_5 = {'PassengerId': test.PassengerId, 'Survived': y_hat_vc_xgb}
submission_5 = pd.DataFrame(data=final_data_5)

final_data_comp = {'PassengerId': test.PassengerId, 'Survived_vc_hard': y_hat_vc_hard, 'Survived_rf': y_hat_rf, 'Survived_vc_soft' : y_hat_vc_soft, 'Survived_vc_all' : y_hat_vc_all,  'Survived_vc_xgb' : y_hat_vc_xgb}
comparison = pd.DataFrame(data=final_data_comp)

NameError: name 'test' is not defined

In [53]:
#track differences between outputs 
comparison['difference_rf_vc_hard'] = comparison.apply(lambda x: 1 if x.Survived_vc_hard != x.Survived_rf else 0, axis =1)
comparison['difference_soft_hard'] = comparison.apply(lambda x: 1 if x.Survived_vc_hard != x.Survived_vc_soft else 0, axis =1)
comparison['difference_hard_all'] = comparison.apply(lambda x: 1 if x.Survived_vc_all != x.Survived_vc_hard else 0, axis =1)

NameError: name 'comparison' is not defined

In [54]:
comparison.difference_hard_all.value_counts()

NameError: name 'comparison' is not defined

In [55]:
#prepare submission files 
submission.to_csv('submission_rf.csv', index =False)
submission_2.to_csv('submission_vc_hard.csv',index=False)
submission_3.to_csv('submission_vc_soft.csv', index=False)
submission_4.to_csv('submission_vc_all.csv', index=False)
submission_5.to_csv('submission_vc_xgb2.csv', index=False)

NameError: name 'submission' is not defined